In [ ]:
from datetime import datetime
import time
import requests
import pandas as pd
import pyotp
import pytz
import threading
import time 

from smartapi import SmartConnect


In [ ]:
START_TIME = (15,20,0)   # Place straddle at 9:20  then place SL of 20% on both CE /PE Leg
LOT = 1
SYMBOL = 'BANKNIFTY'
SL =30
SL_LIMIT = 1
ORDER_TYPE ='INTRADAY' 
api_key = "bTS72pmy"


In [1]:
def login():
    obj=SmartConnect(api_key=apikey)
    data = obj.generateSession(username,pwd,pyotp.TOTP(token).now())
    refreshToken= data['data']['refreshToken']
    profile = obj.getProfile(refreshToken)
    logger.info(f'{obj.rmsLimit()}')
    return obj


def getSymbolDf():
    url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'
    d = requests.get(url).json()
    token_df = pd.DataFrame.from_dict(d)
    token_df['expiry'] = pd.to_datetime(token_df['expiry']).apply(lambda x: x.date())

    spotSymInfo = token_df[(token_df.symbol == SYMBOL) & (token_df.exch_seg == 'NSE')].iloc[0].to_dict()
    logger.info(f'spotSymInfo : {spotSymInfo}')

    token_df = token_df.astype({'strike': float})
    expiryList  = token_df[(token_df.name ==SYMBOL) & (token_df.instrumenttype == 'OPTIDX')]['expiry'].unique().tolist()
    expiryList.sort()
    recentExpiry= expiryList[0]
    logger.info(f'recentExpiry {recentExpiry}')
    symbolDf = token_df[(token_df.name ==SYMBOL) & (token_df.expiry == recentExpiry) &   (token_df.instrumenttype == 'OPTIDX')]
    return symbolDf, spotSymInfo

def truncate( f):
    if f is None: return None
    ticksize = 0.05*100
    remainder = int(str(f*100.0).split('.')[0][-2:])
    pp = int((int(remainder/ticksize)*ticksize))

    if len(str(pp))  == 1:
        return float(str(int(f)) + '.0' + str(pp))
    else:
        return float(str(int(f)) + '.' + str(pp))


def getOrderbook():
    for i in range(1,4):
        try:
            orderbookRes = SMARTAPI_OBJ.orderBook() 
            if 'data' in orderbookRes and orderbookRes['data']:
                orderDf = pd.DataFrame(orderbookRes['data'])
                return orderDf
        except Exception as e:
            logger.exception(f'Orderbook  Failed') 
        time.sleep(i*2)
      
    

def getExecutedPrice(orderid):
    executedPrice = 0
    for i in range(1,5):
        try:
            orderdf = getOrderbook()
            orderInfo  = orderdf[(orderdf.orderid == str(orderid))]
            if int(orderInfo['filledshares'])  == int(orderInfo['quantity']):
                return float(orderInfo['averageprice'])
        except Exception as e:
            logger.error(f'Get Entry Price Failed {orderid}') 
        time.sleep(i)
    return executedPrice

def placeOrder(token, symbol, qty, buy_sell,variety = 'NORMAL',ordertype ='MARKET',triggerprice = 0,price = 0):
        try:
            orderparams = {
                "variety": variety,
                "tradingsymbol": symbol,
                "symboltoken": token,
                "transactiontype": buy_sell,
                "exchange": 'NFO',
                "ordertype": ordertype,
                "producttype": ORDER_TYPE,
                "duration": "DAY",
                "price": truncate(price),
                "squareoff": "0",
                "stoploss": "0",
                "quantity": qty,
                "triggerprice":truncate(triggerprice),
                "ordertag":'DFT'

            }
            orderId =  SMARTAPI_OBJ.placeOrder(orderparams)
            logger.info(f"Order placed for {orderparams}  order id: {orderId}")
            return orderId
        except Exception as e:
            logger.exception(f"Order placement failed: {e} {orderparams}")

    
def modify_order(orderJson,triggerPrice):
    
    try:
        price = triggerPrice + SL_LIMIT   if orderJson['transactiontype'] == 'BUY' else triggerPrice - SL_LIMIT 
        orderparams = {
        "variety":orderJson['variety'],
        "orderid":orderJson['orderid'],
        "ordertype":orderJson['ordertype'],
        "producttype":orderJson['producttype'],
        "triggerprice":truncate(triggerPrice),
        'price':truncate(price),
        "quantity":orderJson['quantity'],
        "duration":"DAY",
        "tradingsymbol":orderJson['tradingsymbol'],
        "symboltoken":orderJson['symboltoken'],
        "exchange":orderJson['exchange']   
        }
        res = SMARTAPI_OBJ.modifyOrder(orderparams)
        logger.info(f"Order modified for {orderJson['tradingsymbol']} {res} {triggerPrice} ")
        
    except Exception as e:
        logger.exception(f'Order modification process Failed  {orderJson} {triggerPrice}') 
        


def startProcess(symbolDf, spotSymInfo):
    ltpInfo = SMARTAPI_OBJ.ltpData('NSE',spotSymInfo['symbol'],spotSymInfo['token'])
    indexLtp = ltpInfo['data']['ltp']
    logger.info(f'spotLtp : {indexLtp}')

    
    atmStrike = int(indexLtp/100)*100  if SYMBOL == 'BANKNIFTY' else int(indexLtp/50)*50   # calculate atm  as per your logic
    logger.info(f'atm : {atmStrike}')
    ceSymbol  =  symbolDf[ (symbolDf.strike == atmStrike*100) &  (symbolDf.symbol.str.endswith('CE')) ].iloc[0]
    peSymbol  =  symbolDf[ (symbolDf.strike == atmStrike*100) & (symbolDf.symbol.str.endswith('PE') )].iloc[0]
    logger.info(f'ceSymbol : {ceSymbol.to_dict()}')
    logger.info(f'peSymbol : {peSymbol.to_dict()}')
    transType = 'SELL'   # BUY change as per ur need 

    ceOrderid = placeOrder(ceSymbol['token'],ceSymbol['symbol'],LOT*int(ceSymbol['lotsize']),transType)
    peOrderid = placeOrder(peSymbol['token'],peSymbol['symbol'],LOT*int(peSymbol['lotsize']),transType)
        

    # PLACE SL ORDER  
    ceEntryPrice =  getExecutedPrice(ceOrderid)
    peEntryPrice = getExecutedPrice(peOrderid)
    logger.info(f'Entry Price CE   : {ceEntryPrice}  PE:  {peEntryPrice}')

    if ceEntryPrice > 0:
        slTriggerPice  = ceEntryPrice*(1 - SL/100)  if transType == 'BUY' else  ceEntryPrice*(1 + SL/100)
        slTransType = 'BUY' if transType == 'SELL' else 'SELL'
        slLimit = slTriggerPice + SL_LIMIT if transType == 'SELL' else slTriggerPice - SL_LIMIT
        ceslorderid = placeOrder(ceSymbol['token'],ceSymbol['symbol'],LOT*int(ceSymbol['lotsize']),slTransType ,variety='STOPLOSS',ordertype= 'STOPLOSS_LIMIT', triggerprice = slTriggerPice, price = slLimit)

        
    if peEntryPrice > 0:
        slTriggerPice  = peEntryPrice*(1 - SL/100)  if transType == 'BUY' else  peEntryPrice*(1 + SL/100)
        slTransType = 'BUY' if transType == 'SELL' else 'SELL'
        slLimit = slTriggerPice + SL_LIMIT if transType == 'SELL' else slTriggerPice - SL_LIMIT
        peslorderid =placeOrder(peSymbol['token'],peSymbol['symbol'],LOT*int(peSymbol['lotsize']),slTransType ,variety='STOPLOSS',ordertype= 'STOPLOSS_LIMIT', triggerprice = slTriggerPice, price = slLimit)

    # CHECK SL
    endTime = datetime.now(pytz.timezone('Asia/Kolkata')).replace(hour=15, minute=0,second=0)
    while datetime.now(pytz.timezone('Asia/Kolkata')) <  endTime and ceEntryPrice > 0 and  peEntryPrice > 0:
        try:
            orderDf = getOrderbook()
            ceSLorderInfo = orderDf[orderDf.orderid == str(ceslorderid)].iloc[0]
            peSLorderInfo = orderDf[orderDf.orderid == str(peslorderid)].iloc[0]

            if int(ceSLorderInfo['filledshares'])  == int(ceSLorderInfo['quantity']): 
                logger.info(f'SL hit on  CE side')
                modify_order(peSLorderInfo,peEntryPrice)
                break

            if int(peSLorderInfo['filledshares'])  == int(peSLorderInfo['quantity']): 
                logger.info(f'SL hit on  PE side')
                modify_order(ceSLorderInfo,ceEntryPrice)
                break
        except Exception :
            logger.exception(f'Error is SL order check')  
        time.sleep(5)

        



In [ ]:
if __name__ == '__main__':
    SMARTAPI_OBJ  = login()
    symbolDf, spotSymInfo = getSymbolDf()
   
    startTime =  datetime.now(pytz.timezone('Asia/Kolkata'))
    closingTime = startTime.replace(hour=START_TIME[0], minute=START_TIME[1],second=START_TIME[2])
    interval = max(0,(closingTime - startTime).total_seconds())
    logger.info(f'Entry  will Start after  {interval} sec' )
    time.sleep(interval)
    startProcess( symbolDf, spotSymInfo)
